In [28]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests

In [2]:
pip install cassandra-driver


Note: you may need to restart the kernel to use updated packages.


Connect to Cassandra

In [14]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

Create keyspace

In [6]:
keyspace_name = "data_quran"  


session.execute("CREATE KEYSPACE IF NOT EXISTS data_quran WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.set_keyspace('data_quran')

use keyspace

In [25]:
session.set_keyspace('data_quran')

create table

In [26]:
table_name = "quran_verses"


In [31]:
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        surah_id INT,
        verse_id INT,
        verse_text TEXT,
        PRIMARY KEY (surah_id, verse_id)
    )
"""
session.execute(create_table_query)

In [36]:
table_name2 = "quran_translation"
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name2} (
        surah_id INT,
        verse_id INT,
        translation_text TEXT,
        language TEXT,
        PRIMARY KEY ((surah_id, verse_id), language)
    )
"""
session.execute(create_table_query)

Extract data from API

In [32]:
api_url = "http://api.alquran.cloud/v1/quran/ar.alafasy"
response = requests.get(api_url)
data = response.json()


In [33]:
for surah in data['data']['surahs']:
    surah_id = surah['number']
    for verse in surah['ayahs']:
        verse_id = verse['number']
        verse_text = verse['text']
        insert_query = f"""
            INSERT INTO {table_name} (surah_id, verse_id, verse_text)
            VALUES (%s, %s, %s)
        """
        session.execute(insert_query, (surah_id, verse_id, verse_text))

In [35]:
select_query = f"SELECT * FROM {table_name} LIMIT 2"
rows = session.execute(select_query)


for row in rows:
    print(row)


Row(surah_id=23, verse_id=2674, verse_text='بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ قَدْ أَفْلَحَ ٱلْمُؤْمِنُونَ')
Row(surah_id=23, verse_id=2675, verse_text='ٱلَّذِينَ هُمْ فِى صَلَاتِهِمْ خَٰشِعُونَ')


In [37]:
api_url2 = "http://api.alquran.cloud/v1/edition/en.sahih"
response2 = requests.get(api_url)
data2 = response.json()

In [38]:
for surah in data['data']['surahs']:
    surah_id = surah['number']
    for verse in surah['ayahs']:
        verse_id = verse['number']
        verse_text = verse['text']
        insert_query = f"""
            INSERT INTO {table_name2} (surah_id, verse_id, translation_text, language)
            VALUES (%s, %s, %s, %s)
        """
        session.execute(insert_query, (surah_id, verse_id, verse_text, 'english'))

In [42]:
api_url3 = "http://api.alquran.cloud/v1/edition/en.sahih"
response3 = requests.get(api_url)
data3 = response.json()

In [43]:
for surah in data3['data']['surahs']:
    surah_id = surah['number']
    for verse in surah['ayahs']:
        verse_id = verse['number']
        verse_text = verse['text']
        insert_query = f"""
            INSERT INTO {table_name2} (surah_id, verse_id, translation_text, language)
            VALUES (%s, %s, %s, %s)
        """
        session.execute(insert_query, (surah_id, verse_id, verse_text, 'arabic'))

In [44]:
select_query = f"SELECT * FROM {table_name2} LIMIT 2"
rows = session.execute(select_query)


print(row)

Row(surah_id=60, verse_id=5159, language='english', translation_text='إِنَّمَا يَنْهَىٰكُمُ ٱللَّهُ عَنِ ٱلَّذِينَ قَٰتَلُوكُمْ فِى ٱلدِّينِ وَأَخْرَجُوكُم مِّن دِيَٰرِكُمْ وَظَٰهَرُوا۟ عَلَىٰٓ إِخْرَاجِكُمْ أَن تَوَلَّوْهُمْ ۚ وَمَن يَتَوَلَّهُمْ فَأُو۟لَٰٓئِكَ هُمُ ٱلظَّٰلِمُونَ')
